In [1]:
from tempfile import mkdtemp
import codecs

import os
import re
import time
import datetime
import pandas as pd
import urllib.request
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.select import Select

import googleapiclient.errors
from googleapiclient.discovery import build

In [2]:
# パラメータ設定
DATABASE_URL    = "https://vtuber-post.com/database.html"
output_folder        = "..\csv"

In [3]:
# ファイル関連

In [5]:
# ブラウザの起動
options = webdriver.ChromeOptions()
options.add_argument("--window-size=1200,1000")
driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver=driver, timeout=30)

In [6]:
list_form = [{
        "id":"gender",
        "feature": ["male", "female", "babiniku", "tomgirl", "bisexual", "other"]
    },{
        "id":"genre",
        "feature": ["sing", "dance", "paint", "craft", "perform", "music", "game", "trpg", "anime", "chat", "physics_calc", "comedy", "cook", "handicraft", "fashion", "animal", "shock", "vehicle", "travel" ,"sports", "martial_arts", "trend", "pachinko", "slot", "mahjong", "how_to", "education", "history", "science", "news", "other"]
    },{
        "id":"model",
        "feature": ["2d", "3d", "real"]
    },{
        "id":"customer",
        "feature": ["illust", "2d_modeling", "3d_modeling", "event", "talk", "comedy", "game", "sing", "perform", "composition", "video_edit", "foreign_lng"]
    },{
        "id":"collabo",
        "feature": [5,4,3,2,1,0]
    }
]

In [9]:
k = 1
while 1:

    form_id = list_form[k]["id"]

    if form_id == "collabo":
        df = pd.DataFrame({
            "channelId"     : [],
            "collabo_level" : []
        })
    else:
        df = pd.DataFrame({
            "channelId" : []
        })

    # 特徴のプルダウンでループ
    for j in range(len(list_form[k]["feature"])):

        feature = list_form[k]["feature"][j]

        # Vtuberデータベースを開く
        driver.get(DATABASE_URL)
        wait.until(EC.presence_of_all_elements_located)

        # 検索条件を入力

        # 所属：所属なし
        office = driver.find_element(By.ID, 'office')
        select_office = Select(office)
        select_office.select_by_index(1)

        # 表示件数：100件
        limit = driver.find_element(By.ID, 'limit')
        select_limit = Select(limit)
        select_limit.select_by_index(2)

        # 検索条件を追加する
        more = driver.find_element(By.XPATH, '//*[@id="cont_search_bottom_wrap"]/dl/div/span')
        more.click()
        time.sleep(3)

        # 特徴を抽出するフォーム
        form_feature = driver.find_element(By.ID, form_id)
        select_form_feature = Select(form_feature)
        select_form_feature.select_by_index(j +1)

        # 上記の条件で絞り込む
        search_submit = driver.find_element(By.ID, 'search_submit')
        search_submit.click()
        time.sleep(3)

        list_channelId = []
        flag_end = 0

        # ページのループ
        page = 0
        while 1:
            page += 1

            # ページの切り替え処理
            if page!=1:
                # 適切なページボタンを探すためのループ
                i = 0
                while 1:
                    i += 1

                    try:
                        button = driver.find_element(By.XPATH, f'//*[@id="cont"]/p[1]/a[{i}]')
                    # 次のページが無ければループを抜ける
                    except NoSuchElementException:
                        flag_end = 1
                        break

                    # ボタンに書かれた数字が次のページ番号ならボタンを押す
                    buttonNum = button.text
                    if buttonNum == str(page):
                        button.click()
                        wait = WebDriverWait(driver=driver, timeout=30)
                        break

            #次のページがなければループを抜ける
            if flag_end:break

            time.sleep(1)

            # ページ内でchannelIDを収集するためのループ
            i = 0
            while 1:
                i += 1

                try:
                    youtubeURL = driver.find_element(By.XPATH, f'//*[@id="cont"]/div/div[{i}]/p[1]/a[1]').get_attribute("href")
                except NoSuchElementException:
                    break

                channelId  = youtubeURL.replace('https://vtuber-post.com/database_detail.html?id=','')
                list_channelId.append(channelId)

        if form_id == "collabo" :
            df_add = pd.DataFrame({
                "channelId" : list_channelId,
                "collabo_level" : feature
            })
            df = pd.concat([df, df_add.drop_duplicates()], axis=0)

        else:
            col_name = f"{form_id}_{feature}"
            # データフレームに読み取った情報を追加
            df_add = pd.DataFrame({
                "channelId" : list_channelId,
                col_name    : 1
            })
            df = df.merge(df_add.drop_duplicates(), on="channelId", how = 'outer')

        #debug:
    
    #debug:

    # csvに保存
    if form_id == "collabo" :
        df = df.astype({f"collabo_level":int})
        df.to_csv(f"../csv2/{form_id}.csv", index = False)
    else:
        df = df.fillna(0)
        for col in list_form[k]["feature"]:
            df = df.astype({f"{form_id}_{col}":int})
        df.to_csv(f"../csv2/{form_id}.csv", index = False)

    k += 1

IndexError: list index out of range